In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import os
import sys
import polars as pl

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

from utilities.load_utils import *
from utilities.model_utils import *

import analysis_utilities_polars as apl
import export_analysis as ea
import pricing.pricing_logic as prl
import api.config as ac

In [3]:
path_manager = PathManager('mubi')
load_manager = LoadManager(path_manager)

In [4]:
api_config = ac.ApiConfig.load_from_json('mubi_cheapest_offers_v2')
on_top = load_manager.load_on_top_file()

In [5]:
models = {}

for target_variable, model_config_name in api_config.target_variables_and_model_config.items():
    model_name = path_manager.get_model_name(api_config.train_data_name, target_variable, model_config_name)
    model = load_manager.load_model(api_config.train_data_name, model_name)
    models[target_variable] = model

In [6]:
data_hu, fi, fo, fm = load_manager.load_data('mubi_v24') 
data_hu = data_hu[fi + fo + fm]

In [7]:
addition_on_exp_drivers = data_hu[data_hu['driver_experience'].between(0, 1)].sample(500)
addition_young_cars = data_hu[data_hu['vehicle_age'] < 2].sample(500)
addition_young_people = data_hu[data_hu['contractor_age'].between(17, 18)].sample(500)
addition_old_people = data_hu[data_hu['contractor_age'].between(70, 90)].sample(500)
addition_warsawa = data_hu[data_hu['county'].astype(str).str.contains('Warszawa')].sample(500)
addition_random = data_hu[data_hu['licence_at_age'] == 18].sample(50000)

addition = pd.concat(
    [
     addition_on_exp_drivers,
     addition_young_cars,
     addition_young_people,
     addition_old_people,
     addition_warsawa,
     addition_random
    ]).drop_duplicates()
addition['crawling_date'] = '2025.01.01'
addition['policy_start_date'] = None
addition['contractor_birth_date'] = None
addition['contractor_driver_licence_date'] = None

In [8]:
for target_variable, model in models.items():
    addition[target_variable] = predict_on_top(model, addition[model.feature_names], on_top, target_variable)
for target_variable in models.keys():
    addition[target_variable + "_model_prediction"] = addition[target_variable]
    
addition['data_source'] = 'hungary_quotes'

In [9]:
data_pl, fi, fo, fm = load_manager.load_data('mubi_v23') 
data_pl = data_pl[fi + fo + fm + list(api_config.target_variables_and_model_config.keys())]
data_pl = data_pl[~((data_pl['vehicle_maker'] == 'FORD') & (data_pl['vehicle_model'] == 'Fiesta') & (data_pl['vehicle_infoexpert_version'].isna()) & (data_pl['contractor_birth_date'].astype(str) == '1984.01.01'))]
data_pl = data_pl[~data_pl['UNIQA-(OC),(NNW),(Assistance=75 km PL,After breakdown,Replacement vehicle)-price'].isnull()]

In [10]:
for target_variable in api_config.target_variables_and_model_config:
    model_features = models[target_variable].feature_names
    data_pl[target_variable + '_model_prediction'] = predict_on_top(models[target_variable], data_pl[model_features], on_top, target_variable)
data_pl['data_source'] = 'crawled_data'

In [11]:
data_dtypes = dict(zip(data_pl.columns, data_pl.dtypes))
sim_data = pd.concat([data_pl, addition])
sim_data = sim_data.astype(data_dtypes)

In [12]:
sim_data[sim_data['data_source'] == 'crawled_data'].filter(like = 'MTU')

,MTU24-(OC)-price,MTU24-(OC)-price_model_prediction
0,928.0,925.534082
2,1349.0,1349.016598
3,1339.0,1358.041608
4,873.0,860.729625
5,1215.0,1255.552385
...,...,...
15401,842.0,870.132627
15402,1325.0,1326.706183
15403,1614.0,1611.147115
15404,1190.0,1188.245796


In [13]:
gc.collect()

10

In [17]:
api_configs = {
    'base_config' : api_config
}

#for uniqa_weight in [0.5, 0.55, 0.6, 0.65, 0.7, 0.8, 0.9]:
for uniqa_weight in [0.76]:
    for top_k in [3]:
        for undercut_factor in [0.93, 0.95, 0.99]:
            api_config_ = api_config.model_copy(deep=True)
            api_config_.tp_kernel[0]['weight'] = uniqa_weight
            sum_rest = sum([x['weight'] for x in api_config_.tp_kernel[1:]])
            for i in range(1, len(api_config_.tp_kernel)):
                api_config_.tp_kernel[i]['weight'] = round((api_config_.tp_kernel[i]['weight'] / sum_rest) * (1 - uniqa_weight), 3)
            api_config_.rank1_undercut_factor = undercut_factor
            api_config_.tp_take_top_k = top_k
            api_configs[f'Uniqa weight={uniqa_weight},Undercut Factor={undercut_factor},Take TP top {top_k}'] = api_config_

In [20]:
api_configs[f'Uniqa weight=0.76,Undercut Factor=0.93,Take TP top 3'].tp_kernel

[{'target_variable': 'UNIQA-(OC),(NNW),(Assistance=75 km PL,After breakdown,Replacement vehicle)-price',
  'cost_estimate': 0.93,
  'weight': 0.76},
 {'target_variable': 'MTU24-(OC)-price',
  'cost_estimate': 0.89,
  'weight': 0.16},
 {'target_variable': 'ALLIANZ-(OC)-price',
  'cost_estimate': 0.89,
  'weight': 0.04},
 {'target_variable': 'LINK4-(OC),(Assistance=100 km PL,Replacement vehicle)-price',
  'cost_estimate': 0.87,
  'weight': 0.04}]

In [34]:
api_config = ac.ApiConfig.load_from_json('mubi_cheapest_offers_v2')

In [35]:
price_cols_wo_ominimo = sorted([col for col in data_pl if col.endswith('-price')])
insurers_wo_ominimo = sorted([col.split('-')[0] for col in price_cols_wo_ominimo])

In [37]:
ba_hu = apl.create_basic_analysis(pl.from_pandas(sim_data).filter(pl.col('data_source') == 'hungary_quotes'), insurers_wo_ominimo, price_cols_wo_ominimo, has_conversion_data=False, country='pl').to_pandas()

In [38]:
ba_pl = apl.create_basic_analysis(pl.from_pandas(sim_data).filter(pl.col('data_source') == 'crawled_data'), insurers_wo_ominimo, price_cols_wo_ominimo, has_conversion_data=False, country='pl').to_pandas()

In [19]:
ba_hu.iloc[:, 2:] - ba_pl.iloc[:, 2:]

,ALLIANZ,BALCIA,BEESAFE,BENEFIA,ERGOHESTIA,GENERALI,LINK4,MTU24,PROAMA,TRASTI,TUZ,UNIQA,WEFOX
0,0.7600,0.0200,0.0100,0.3300,0.0800,0.0100,0.0600,0.1500,0.0100,0.0900,0.0400,0.0000,0.0200
1,609.1447,261.3343,201.5999,882.3675,219.3800,309.4930,240.6501,607.1348,448.6177,287.5018,2484.8344,350.4588,1235.4672
2,632.5408,314.8724,NaN,422.3254,399.4270,68.4465,255.7282,79.5926,NaN,240.5822,NaN,187.8013,NaN
3,1.0521,0.5001,0.8439,1.3665,-0.5815,0.8670,-0.4236,1.7223,0.9538,-0.1001,1.6920,0.2638,1.4657
4,-0.0757,-0.3823,NaN,-0.2242,8.6796,-0.1500,3.4520,-3.9289,NaN,-0.7457,NaN,-6.6630,NaN
5,-0.1487,-0.5079,0.0015,-0.2634,3.1717,-0.6235,16.4513,-26.4201,-0.0220,7.9931,NaN,0.3681,NaN
6,-0.4872,-1.0649,0.0334,-0.0682,0.4232,-1.2071,-3.3334,-1.2761,-0.3295,5.4391,NaN,1.8708,NaN
7,-113.3740,-41.5670,NaN,52.4176,220.5106,66.0347,35.8334,37.9597,NaN,-50.7312,NaN,46.7659,NaN
8,-48.8470,-141.8440,74.4365,-60.0682,3.3950,65.0974,38.7791,-36.5216,-35.9164,13.6437,NaN,-20.4072,NaN
9,-109.6869,-192.5725,-62.4600,-105.1646,-46.0481,72.8460,19.0934,-48.6775,-98.5408,-1.6853,NaN,-67.7871,NaN


In [39]:
apl.create_basic_analysis(pl.from_pandas(sim_data), insurers_wo_ominimo, price_cols_wo_ominimo, has_conversion_data=False, country='pl').to_pandas()

,Insurer,Whole Market,ALLIANZ,BALCIA,BEESAFE,BENEFIA,ERGOHESTIA,GENERALI,LINK4,MTU24,PROAMA,TRASTI,TUZ,UNIQA,WEFOX
0,share_of_quotes,NaN,0.8600,1.0000,1.0000,0.9400,0.9900,1.0000,0.9900,0.9700,1.0000,0.9800,0.9900,1.0000,1.0000
1,mean_prices_overall,NaN,2509.7560,2438.5156,3000.6976,3567.8973,1504.4151,2336.4373,1581.7817,1671.0247,2454.2967,1638.0428,7133.2895,1355.7301,5533.9922
2,mean_prices_sold,NaN,1418.7710,1125.1498,300.7352,1105.6193,1588.2016,983.5539,1355.7140,1119.4410,726.4897,1445.9894,NaN,1016.1259,NaN
3,mean_rank,NaN,7.1967,7.4104,9.5257,10.3516,3.2534,6.7882,3.1929,3.9238,7.8038,3.7094,12.5322,1.8943,11.8560
4,share_of_rank_1,NaN,0.0201,0.1626,0.0015,0.0465,13.0782,0.1146,7.0293,6.7923,0.0294,3.3451,NaN,69.3804,NaN
5,share_of_rank_2,NaN,0.1239,0.5250,0.0093,0.0635,20.1307,0.6024,30.0621,20.8539,0.2509,15.7140,NaN,11.6643,NaN
6,share_of_rank_3,NaN,0.7046,1.1568,0.0434,0.1239,22.4645,1.4325,23.0375,18.1980,0.8812,26.4042,NaN,5.5534,NaN
7,rank_distance_1_to_2,NaN,130.8068,89.1421,19.7258,118.6802,357.1572,40.1562,112.8129,130.5621,55.4949,100.5590,NaN,267.7204,NaN
8,rank_distance_2_to_3,NaN,91.5973,107.4394,96.9304,101.3709,145.5459,44.5766,154.3061,161.8491,65.5241,206.2049,NaN,180.2770,NaN
9,rank_distance_3_to_4,NaN,93.2198,120.6649,141.3015,103.0875,120.6161,54.8944,154.4304,157.4431,88.1134,133.9730,NaN,164.9268,NaN


In [58]:
bas = {
    'Without Ominimo' : apl.create_basic_analysis(pl.from_pandas(sim_data), insurers_wo_ominimo, price_cols_wo_ominimo, has_conversion_data=False, country='pl').to_pandas()
}

for name, current_api_config in list(api_configs.items())[:]:
    
    calc_prices = prl.calculate_price(sim_data.copy(), current_api_config.tp_kernel, current_api_config.rank1_undercut_factor, current_api_config.tp_take_top_k).rename(columns = {'market_price' : "OMINIMO-(OC)-price"})
    price_cols = sorted([col for col in calc_prices if col.endswith('-price')])
    insurers = sorted([col.split('-')[0] for col in price_cols]) 
    
    ba = apl.create_basic_analysis(pl.from_pandas(calc_prices), insurers, price_cols, has_conversion_data = False, country = 'pl')
    ba_crawled = apl.create_basic_analysis(pl.from_pandas(calc_prices).filter(pl.col('data_source') == 'crawled_data'), insurers, price_cols, has_conversion_data = False, country = 'pl')
    
    bas[name] = ba.to_pandas()
    
    bas[name + ", crawled_data_only"] = ba_crawled.to_pandas()
    
    gc.collect()
    

In [59]:

reversed_dict = {
    "Contractor Age : Under 18" : True,
    "Contractor Age : 18-24" : True,
    "Contractor Age : Over 75" : True,
    "Number of Damages : 1 damage" : True,
    "Number of Damages : 2 damages" : True,
    "Number of Damages : 3 damages" : True,
    "Driver Experience : 0" : True,
    "Driver Experience : 1" : True,
    "Vehicle Power : 201+" : True,
    "Postal Code Population Density : Dense" : True
}
ea.export_workbook(bas, "sim_data.xlsx", reversed_dict)

/home/luka-erceg/luka_posao/market_models/venv/lib/python3.12/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/luka-erceg/luka_posao/market_models/venv/lib/python3.12/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/luka-erceg/luka_posao/market_models/venv/lib/python3.12/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/luka-erceg/luka_posao/market_models/venv/lib/python3.12/site-packag

In [50]:
bas['base_config']

,Insurer,Whole Market,ALLIANZ,BALCIA,BEESAFE,BENEFIA,ERGOHESTIA,GENERALI,LINK4,MTU24,OMINIMO,PROAMA,TRASTI,TUZ,UNIQA,WEFOX
0,share_of_quotes,NaN,0.8600,1.0000,1.0000,0.9400,0.9900,1.0000,0.9900,0.9700,1.0000,1.0000,0.9800,0.9900,1.0000,1.0000
1,mean_prices_overall,NaN,2509.7560,2438.5156,3000.6976,3567.8973,1504.4151,2336.4373,1581.7817,1671.0247,1499.2878,2454.2967,1638.0428,7133.2895,1355.7301,5533.9922
2,mean_prices_sold,NaN,1451.0022,1127.7832,300.7352,1128.8363,1596.8538,979.6336,1364.1685,1125.7882,1087.9267,726.4897,1455.3786,NaN,1002.7484,NaN
3,mean_rank,NaN,8.1587,8.3530,10.4882,11.3331,4.0084,7.7229,3.9470,4.7383,2.8810,8.7492,4.5233,13.5258,2.1372,12.8497
4,share_of_rank_1,NaN,0.0155,0.1518,0.0015,0.0418,12.5300,0.1100,6.1527,5.4651,12.6198,0.0294,3.0586,NaN,59.8238,NaN
5,share_of_rank_2,NaN,0.1084,0.2571,0.0062,0.0294,7.2786,0.2880,8.8195,5.5581,53.1213,0.1285,8.1985,NaN,16.2065,NaN
6,share_of_rank_3,NaN,0.3345,0.7588,0.0341,0.0542,16.6834,0.9509,26.6923,20.0331,13.8030,0.3872,12.4123,NaN,7.8562,NaN
7,rank_distance_1_to_2,NaN,144.5970,85.3220,19.7258,118.7130,364.8370,41.3725,109.1788,128.0861,43.0886,54.9161,101.4907,NaN,125.4506,NaN
8,rank_distance_2_to_3,NaN,89.2614,88.8098,45.3635,114.6232,127.0295,32.5994,84.9083,125.2103,188.7485,66.9154,313.7239,NaN,129.4634,NaN
9,rank_distance_3_to_4,NaN,118.1943,135.7306,168.9741,113.1547,114.5892,54.5893,159.0401,160.9653,163.6803,84.0764,112.4734,NaN,138.8405,NaN


In [51]:
bas['base_config, crawled_data_only']

,Insurer,Whole Market,ALLIANZ,BALCIA,BEESAFE,BENEFIA,ERGOHESTIA,GENERALI,LINK4,MTU24,OMINIMO,PROAMA,TRASTI,TUZ,UNIQA,WEFOX
0,share_of_quotes,NaN,0.2400,0.9800,0.9900,0.6700,0.9200,0.9900,0.9400,0.8500,1.0000,0.9900,0.9100,0.9600,1.0000,0.9800
1,mean_prices_overall,NaN,1933.2871,2226.0258,2837.0244,2805.3933,1324.0610,2085.2780,1384.6541,1164.5689,1268.8243,2090.2013,1401.1191,5105.1232,1071.9298,4530.5625
2,mean_prices_sold,NaN,1256.2857,978.4808,NaN,1099.1600,1222.1990,926.4231,1147.0472,1105.6086,940.2138,NaN,1272.8128,NaN,826.2274,NaN
3,mean_rank,NaN,7.0574,7.9367,9.7882,10.1164,4.6155,7.0179,4.4381,3.3680,2.3905,7.9629,4.6858,12.1164,2.0756,11.6326
4,share_of_rank_1,NaN,0.0570,0.4234,NaN,0.2035,4.6654,0.2117,2.4670,6.2612,30.1905,NaN,3.1754,NaN,52.3449,NaN
5,share_of_rank_2,NaN,0.2280,0.6025,0.0081,0.1303,5.1865,0.6595,4.0140,6.5706,52.6624,0.2117,4.5188,NaN,25.2076,NaN
6,share_of_rank_3,NaN,0.7409,1.5144,0.0081,0.2198,15.1767,1.7831,15.8362,40.0830,6.9288,0.7084,7.4011,NaN,9.5994,NaN
7,rank_distance_1_to_2,NaN,187.8831,101.9203,NaN,114.8018,167.5209,0.0000,74.6840,120.2552,51.9981,NaN,153.6308,NaN,82.0493,NaN
8,rank_distance_2_to_3,NaN,112.6679,135.7355,34.9000,127.6279,156.3417,0.0000,85.7855,76.8633,195.2843,100.6717,283.4311,NaN,127.6913,NaN
9,rank_distance_3_to_4,NaN,187.5606,249.2573,307.0000,135.0111,109.4046,1.3900,120.9340,179.1953,217.2139,133.4216,128.0439,NaN,183.1056,NaN


In [52]:
bas['base_config']

,Insurer,Whole Market,ALLIANZ,BALCIA,BEESAFE,BENEFIA,ERGOHESTIA,GENERALI,LINK4,MTU24,OMINIMO,PROAMA,TRASTI,TUZ,UNIQA,WEFOX
0,share_of_quotes,NaN,0.8600,1.0000,1.0000,0.9400,0.9900,1.0000,0.9900,0.9700,1.0000,1.0000,0.9800,0.9900,1.0000,1.0000
1,mean_prices_overall,NaN,2509.7560,2438.5156,3000.6976,3567.8973,1504.4151,2336.4373,1581.7817,1671.0247,1499.2878,2454.2967,1638.0428,7133.2895,1355.7301,5533.9922
2,mean_prices_sold,NaN,1451.0022,1127.7832,300.7352,1128.8363,1596.8538,979.6336,1364.1685,1125.7882,1087.9267,726.4897,1455.3786,NaN,1002.7484,NaN
3,mean_rank,NaN,8.1587,8.3530,10.4882,11.3331,4.0084,7.7229,3.9470,4.7383,2.8810,8.7492,4.5233,13.5258,2.1372,12.8497
4,share_of_rank_1,NaN,0.0155,0.1518,0.0015,0.0418,12.5300,0.1100,6.1527,5.4651,12.6198,0.0294,3.0586,NaN,59.8238,NaN
5,share_of_rank_2,NaN,0.1084,0.2571,0.0062,0.0294,7.2786,0.2880,8.8195,5.5581,53.1213,0.1285,8.1985,NaN,16.2065,NaN
6,share_of_rank_3,NaN,0.3345,0.7588,0.0341,0.0542,16.6834,0.9509,26.6923,20.0331,13.8030,0.3872,12.4123,NaN,7.8562,NaN
7,rank_distance_1_to_2,NaN,144.5970,85.3220,19.7258,118.7130,364.8370,41.3725,109.1788,128.0861,43.0886,54.9161,101.4907,NaN,125.4506,NaN
8,rank_distance_2_to_3,NaN,89.2614,88.8098,45.3635,114.6232,127.0295,32.5994,84.9083,125.2103,188.7485,66.9154,313.7239,NaN,129.4634,NaN
9,rank_distance_3_to_4,NaN,118.1943,135.7306,168.9741,113.1547,114.5892,54.5893,159.0401,160.9653,163.6803,84.0764,112.4734,NaN,138.8405,NaN


In [90]:
sim_data_ranks, rank_counts, rank_percentages, (price_columns, rank_cols, rank_price_cols, rank_distance_cols, rank_won_cols) = apl.rank_analysis(pl.from_pandas(sim_data), price_cols_wo_ominimo, '-price', '-rank')

In [97]:
prl.calculate_conversion_factor(sim_data_ranks, 'rank_1-price', prl.STANDARD_CONVERSION_RULES)

.when([(col("rank_1-price")) <= ([(col("rank_1-price")) * (dyn float: 0.8)])]).then(dyn float: 0.88).otherwise(.when([(col("rank_1-price")) <= ([(col("rank_1-price")) * (dyn float: 0.85)])]).then(dyn float: 0.84).otherwise(.when([(col("rank_1-price")) <= ([(col("rank_1-price")) * (dyn float: 0.9)])]).then(dyn float: 0.8).otherwise(.when([(col("rank_1-price")) <= ([(col("rank_1-price")) * (dyn float: 0.925)])]).then(dyn float: 0.73).otherwise(.when([(col("rank_1-price")) <= ([(col("rank_1-price")) * (dyn float: 0.95)])]).then(dyn float: 0.65).otherwise(.when([(col("rank_1-price")) <= ([(col("rank_1-price")) * (dyn float: 0.96)])]).then(dyn float: 0.6).otherwise(.when([(col("rank_1-price")) <= ([(col("rank_1-price")) * (dyn float: 0.97)])]).then(dyn float: 0.5).otherwise(.when([(col("rank_1-price")) <= ([(col("rank_1-price")) * (dyn float: 0.98)])]).then(dyn float: 0.45).otherwise(.when([(col("rank_1-price")) <= ([(col("rank_1-price")) * (dyn float: 0.99)])]).then(dyn float: 0.4).otherwi

id_case,crawling_date,policy_start_date,contractor_birth_date,contractor_driver_licence_date,contractor_personal_id,contractor_postal_code,vehicle_make_year,vehicle_trim,vehicle_eurotax_version,vehicle_infoexpert_model,vehicle_infoexpert_version,vehicle_type,vehicle_licence_plate,vehicle_engine_size,vehicle_power,vehicle_weight_to_power_ratio,vehicle_net_weight,vehicle_gross_weight,vehicle_number_of_seats,vehicle_number_of_doors,vehicle_age,vehicle_steering_wheel_right,vehicle_imported,vehicle_imported_within_last_12_months,contractor_age,licence_at_age,driver_experience,latitude,longitude,postal_code_population,postal_code_area,postal_code_population_density,contractor_mtpl_policy_years,contractor_mtpl_number_of_claims,contractor_mtpl_years_since_last_damage_caused,contractor_children_under_26,additional_driver_under_26,additional_driver_under_26_license_obtained_year,GENERALI_pesel_ab_test,balcia_vehicle_value,beesafe_vehicle_value,benefia_vehicle_value,ergohestia_vehicle_value,generali_vehicle_value,link4_vehicle_value,mtu24_vehicle_value,proama_vehicle_value,trasti_vehicle_value,tuz_vehicle_value,uniqa_vehicle_value,wefox_vehicle_value,wiener_vehicle_value,ycd_vehicle_value,vehicle_maker,vehicle_fuel_type,vehicle_model,vehicle_parking_place,vehicle_usage,vehicle_first_registration_country,vehicle_planned_annual_mileage,contractor_marital_status,voivodeship,county,ALLIANZ-(OC)-price,BEESAFE-(OC)-price,"TUZ-(OC),(NNW),(Assistance=100 km PL)-price","BENEFIA-(OC),(NNW),(Assistance=150 km EU,After breakdown)-price",BALCIA-(OC)-price,"WEFOX-(OC),(Assistance=150 km PL)-price",ERGOHESTIA-(OC)-price,"UNIQA-(OC),(NNW),(Assistance=75 km PL,After breakdown,Replacement vehicle)-price",TRASTI-(OC)-price,GENERALI-(OC)-price,MTU24-(OC)-price,PROAMA-(OC)-price,"LINK4-(OC),(Assistance=100 km PL,Replacement vehicle)-price",ALLIANZ-(OC)-price_model_prediction,BEESAFE-(OC)-price_model_prediction,"TUZ-(OC),(NNW),(Assistance=100 km PL)-price_model_prediction","BENEFIA-(OC),(NNW),(Assistance=150 km EU,After breakdown)-price_model_prediction",BALCIA-(OC)-price_model_prediction,"WEFOX-(OC),(Assistance=150 km PL)-price_model_prediction",ERGOHESTIA-(OC)-price_model_prediction,"UNIQA-(OC),(NNW),(Assistance=75 km PL,After breakdown,Replacement vehicle)-price_model_prediction",TRASTI-(OC)-price_model_prediction,GENERALI-(OC)-price_model_prediction,MTU24-(OC)-price_model_prediction,PROAMA-(OC)-price_model_prediction,"LINK4-(OC),(Assistance=100 km PL,Replacement vehicle)-price_model_prediction",data_source,rank_1-price,rank_2-price,rank_3-price,rank_4-price,rank_5-price,rank_6-price,rank_7-price,rank_8-price,rank_9-price,rank_10-price,rank_11-price,rank_12-price,rank_13-price,ALLIANZ-(OC)-rank,BALCIA-(OC)-rank,BEESAFE-(OC)-rank,"BENEFIA-(OC),(NNW),(Assistance=150 km EU,After breakdown)-rank",ERGOHESTIA-(OC)-rank,GENERALI-(OC)-rank,"LINK4-(OC),(Assistance=100 km PL,Replacement vehicle)-rank",MTU24-(OC)-rank,PROAMA-(OC)-rank,TRASTI-(OC)-rank,"TUZ-(OC),(NNW),(Assistance=100 km PL)-rank","UNIQA-(OC),(NNW),(Assistance=75 km PL,After breakdown,Replacement vehicle)-rank","WEFOX-(OC),(Assistance=150 km PL)-rank",rank_distance_1_to_2,rank_distance_2_to_3,rank_distance_3_to_4,rank_distance_4_to_5,rank_distance_5_to_6,rank_distance_6_to_7,rank_distance_7_to_8,rank_distance_8_to_9,rank_distance_9_to_10,rank_distance_10_to_11,rank_distance_11_to_12,rank_distance_12_to_13,winning_quote,rank_1_won,rank_2_won,rank_3_won,rank_4_won,rank_5_won,rank_6_won,rank_7_won,rank_8_won,rank_9_won,rank_10_won,rank_11_won,rank_12_won,rank_13_won,conversion_factor
i64,str,str,str,str,i64,str,i64,str,str,str,str,str,str,i64,i64,f64,i64,i64,i64,i64,i64,bool,bool,bool,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,bool,bool,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,cat,cat,cat,cat,cat,cat,i64,cat,cat,cat,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f32,f32,f64,f32,f64,f32,f64,f64,f64,f64,f32,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32,i32,

In [93]:
sim_data_ranks.select('conversion_factor')

ColumnNotFoundError: conversion_factor

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'select' <---
DF ["id_case", "crawling_date", "policy_start_date", "contractor_birth_date"]; PROJECT */143 COLUMNS